In [18]:
!pip install opencorpora-tools
!pip install stanza
!pip install razdel
!pip install corus
!pip install nerus
!pip install natasha

     |████████████████████████████████| 34.4 MB 1.4 MB/s eta 0:00:01     |████████████████████████████▉   | 31.0 MB 1.4 MB/s eta 0:00:03
     |████████████████████████████████| 55 kB 5.0 MB/s  eta 0:00:01
     |████████████████████████████████| 49 kB 8.1 MB/s  eta 0:00:01
     |████████████████████████████████| 41 kB 84 kB/s s eta 0:00:01
     |████████████████████████████████| 8.2 MB 64.2 MB/s eta 0:00:01
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13704 sha256=f17c23c3bc2127c5c4766cb181b3ed29d5b2dd987155574718524fc8ce413e2e
  Stored in directory: /home/kukuruku/.cache/pip/wheels/56/ea/58/ead137b087d9e326852a851351d1debf4ada529b6ac0ec4e8c
  Created wheel for intervaltree: filename=intervaltree-3.1.0-py2.py3-none-any.whl size=26102 sha256=0ffd4e2343d44b154cc4c9aa689374d71c039b7567c765719c6cddbc6fb9163c
  Stored in directory: /home/kukuruku/.cache/pip/wheels/45/23/de/5789a92962483fd33cb06674792b9697c1b3766d7c7742830e
Successfully built docopt intervaltree

In [2]:
import stanza

if not 'nlp' in locals():
    stanza.download('ru')
    nlp = stanza.Pipeline('ru')

2022-02-25 23:16:09 INFO: Downloading default packages for language: ru (Russian)...
2022-02-25 23:16:16 INFO: File exists: /home/kukuruku/stanza_resources/ru/default.zip.
2022-02-25 23:16:24 INFO: Finished downloading models and saved to /home/kukuruku/stanza_resources.
2022-02-25 23:16:24 INFO: Loading these models for language: ru (Russian):
| Processor | Package   |
-------------------------
| tokenize  | syntagrus |
| pos       | syntagrus |
| lemma     | syntagrus |
| depparse  | syntagrus |
| ner       | wikiner   |

2022-02-25 23:16:24 INFO: Use device: cpu
2022-02-25 23:16:24 INFO: Loading: tokenize
2022-02-25 23:16:24 INFO: Loading: pos
2022-02-25 23:16:24 INFO: Loading: lemma
2022-02-25 23:16:25 INFO: Loading: depparse
2022-02-25 23:16:25 INFO: Loading: ner
2022-02-25 23:16:26 INFO: Done loading processors!


In [3]:
from collections import defaultdict

i = 0

rels = defaultdict(list)

for doc in corpus:
    i+= 1
    if i > 10: break
        
    print(f"doc: {i}")
        
    for sent in doc.sentences:  
        for [obj, root] in find_deps(sent.source):
            rels[obj].append(root)
            print(f"rels: {len(rels)}")


def find_deps(text: str):
    result = []
    doc = nlp(text)
    
    for sent in doc.sentences:
        for word in sent.words:
            if word.deprel == 'obj':
                result.append((word.lemma, sent.words[word.head - 1].lemma))
                
    return result

rels

NameError: name 'corpus' is not defined

In [4]:
from razdel import sentenize

list(sentenize('Моросил скучный дождь, небо было серым и холодным, стены домов мокрыми и облезлыми'))

[Substring(0,
           82,
           'Моросил скучный дождь, небо было серым и холодным, стены домов мокрыми и облезлыми')]

In [20]:
%time nlp('белая курица')

CPU times: user 66.4 ms, sys: 3.44 ms, total: 69.9 ms
Wall time: 69 ms


[
  [
    {
      "id": 1,
      "text": "белая",
      "lemma": "белый",
      "upos": "ADJ",
      "feats": "Case=Nom|Degree=Pos|Gender=Fem|Number=Sing",
      "head": 2,
      "deprel": "amod",
      "start_char": 0,
      "end_char": 5,
      "ner": "O"
    },
    {
      "id": 2,
      "text": "курица",
      "lemma": "курица",
      "upos": "NOUN",
      "feats": "Animacy=Inan|Case=Nom|Gender=Fem|Number=Sing",
      "head": 0,
      "deprel": "root",
      "start_char": 6,
      "end_char": 12,
      "ner": "O"
    }
  ]
]

In [7]:
from natasha import (
    Segmenter,
    MorphVocab,
    
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,
    
    PER,
    NamesExtractor,

    Doc
)


segmenter = Segmenter()
morph_vocab = MorphVocab()

emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)
ner_tagger = NewsNERTagger(emb)

names_extractor = NamesExtractor(morph_vocab)

text = 'Посол Израиля на Украине Йоэль Лион признался, что пришел в шок, узнав о решении властей Львовской области объявить 2019 год годом лидера запрещенной в России Организации украинских националистов (ОУН) Степана Бандеры. Свое заявление он разместил в Twitter. «Я не могу понять, как прославление тех, кто непосредственно принимал участие в ужасных антисемитских преступлениях, помогает бороться с антисемитизмом и ксенофобией. Украина не должна забывать о преступлениях, совершенных против украинских евреев, и никоим образом не отмечать их через почитание их исполнителей», — написал дипломат. 11 декабря Львовский областной совет принял решение провозгласить 2019 год в регионе годом Степана Бандеры в связи с празднованием 110-летия со дня рождения лидера ОУН (Бандера родился 1 января 1909 года). В июле аналогичное решение принял Житомирский областной совет. В начале месяца с предложением к президенту страны Петру Порошенко вернуть Бандере звание Героя Украины обратились депутаты Верховной Рады. Парламентарии уверены, что признание Бандеры национальным героем поможет в борьбе с подрывной деятельностью против Украины в информационном поле, а также остановит «распространение мифов, созданных российской пропагандой». Степан Бандера (1909-1959) был одним из лидеров Организации украинских националистов, выступающей за создание независимого государства на территориях с украиноязычным населением. В 2010 году в период президентства Виктора Ющенко Бандера был посмертно признан Героем Украины, однако впоследствии это решение было отменено судом. '
doc = Doc(text)

In [19]:
def nerus_f():
    text = 'Посол Израиля на Украине Йоэль Лион признался, что пришел в шок, узнав о решении властей Львовской области объявить 2019 год годом лидера запрещенной в России Организации украинских националистов (ОУН) Степана Бандеры. Свое заявление он разместил в Twitter. «Я не могу понять, как прославление тех, кто непосредственно принимал участие в ужасных антисемитских преступлениях, помогает бороться с антисемитизмом и ксенофобией.'
    doc = Doc(text)
    doc.segment(segmenter)
    doc.tag_morph(morph_tagger)
    doc.parse_syntax(syntax_parser)
    return doc.tokens


%time nerus_f()[6]

CPU times: user 27 ms, sys: 2.7 ms, total: 29.7 ms
Wall time: 19 ms


DocToken(start=36, stop=45, text='признался', id='1_7', head_id='1_0', rel='root', pos='VERB', feats=<Perf,Masc,Ind,Sing,Past,Fin,Mid>)